# Imports

In [ ]:
import itertools as it
import os
import re
from collections import namedtuple
from functools import partial
from pathlib import Path
import pickle
import dask
import distributed
import h5py
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import skimage.measure
import zarr
from dask import delayed
from dask_jobqueue import SLURMCluster
from distributed import Client, LocalCluster, progress
from tqdm.auto import tqdm

IDX = pd.IndexSlice
tqdm.pandas()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.image_analysis.new as new
from paulssonlab.image_analysis import *

In [ ]:
# %load_ext pyinstrument

In [ ]:
hv.extension("bokeh")

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="06:00:00",
    memory="2GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/log",
    cores=1,
    processes=1,
)
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(1)

In [ ]:
cluster.adapt(maximum=20)

# Handler

In [ ]:
segmentation_channel = "RFP-Penta"
trench_detection_channel = segmentation_channel  # channel for trench detection, almost always same as segmentation_channel
measure_channels = ["RFP-Penta", "YFP-DUAL"]
fish_channels = ["RFP-Penta", "Cy5-PENTA", "Cy7"]

## Load outputs from pickle

In [ ]:
# pickle_filename = "/home/jqs1/group/221108rbsdeglibrary_1.pickle"
pickle_filename = "/home/jqs1/group/221108rbsdeglibrary_1_table.pickle"

In [ ]:
%%time
with open(pickle_filename, "rb") as f:
    table, array = pickle.load(f)

# Helper functions

In [ ]:
def _rename_column(col):
    if col[0] == "mask_measurements":
        return col[1]
    elif col[0] == "measurements":
        return "/".join(col[1:])
    else:
        return "/".join(col)


def reformat_table(table, flatten_column_names=False):
    prefixes = sorted(set(k[0] for k in table.keys()))
    df = pd.concat(
        {
            prefix: pd.concat(
                {
                    k[1:]: pd.concat(table[k], names=["roi"])
                    for k in table.keys()
                    if k[0] == prefix
                },
                names=["fov", "t", "channel"],
            ).unstack("channel")
            for prefix in prefixes
        },
        axis=1,
    )
    if flatten_column_names:
        # replace MultiIndex with Index of slash-separated names like "GFP-PENTA/mean_intensity"
        df.columns = [_rename_column(col) for col in df.columns.values]
    return df

In [ ]:
%%pyinstrument
reformat_table(
    {
        k: v
        for k, v in table.items()
        if k[0] in ("measurements", "mask_measurements") and k[2] < 3
    },
    flatten_column_names=True,
)

In [ ]:
def stack_crops(array, prefix, fov, channel):
    keys = sorted(
        [
            k
            for k in array.keys()
            if len(k) == 4 and k[:2] == (prefix, fov) and k[3] == channel
        ]
    )
    trenches = reduce(operator.and_, [array[k].keys() for k in keys])
    crops = {}
    for trench in list(trenches):
        crops[trench] = np.stack([array[k][trench] for k in keys])
    return crops


def unstack(ary):
    return np.swapaxes(ary, 0, 1).reshape(ary.shape[1], -1)


def pad_and_stack(arys, fill_value=0):
    shape = np.max([ary.shape for ary in arys], axis=0)
    return np.stack(
        [
            np.pad(
                ary,
                ((shape[0] - ary.shape[0], 0), (shape[1] - ary.shape[1], 0)),
                constant_values=fill_value,
            )
            for ary in arys
        ]
    )


def pad_unstack(arys):
    return unstack(pad_and_stack(arys))

# Reformat table

In [ ]:
%%time
all_measurements = reformat_table(
    {
        k: v
        for k, v in table.items()
        if k[0] in ("measurements", "mask_measurements")  # and k[2] < 3
    },
    flatten_column_names=True,
)

In [ ]:
trenches = pd.concat(
    {k[1]: v for k, v in table.items() if k[0] == "trenches"}, names=["fov", "roi"]
)

In [ ]:
all_measurements

# Fix ROI orientation

In [ ]:
%%time
# labeling for odd trench_sets need to be inverted
def fix_label_order(df):
    if trenches["trench_set"].loc[df.index[0][:2]] % 2 == 0:
        return df
    else:
        df["label"] = df["label"].max() - df["label"] + df["label"].min()
        return df.sort_values("label")


all_measurements2 = (
    all_measurements.reset_index(["label"])
    .groupby(["fov", "roi", "t"], group_keys=False)
    .progress_transform(fix_label_order)
).set_index("label", append=True)

In [ ]:
all_measurements2.loc[IDX[:, :, 1, 1]].hvplot.scatter("t", "centroid-0")

In [ ]:
all_measurements2.loc[IDX[:, :, 1, 1]].hvplot.scatter("t", "axis_major_length")

In [ ]:
trenches[trenches["trench_set"] == 2]

In [ ]:
all_measurements.loc[IDX[:, :, 300, 1]].hvplot.scatter("t", "axis_major_length")

In [ ]:
all_measurements2.loc[IDX[:, :, 300, 1]].hvplot.scatter("t", "axis_major_length")

In [ ]:
all_measurements2

In [ ]:
pickle_filename = "/home/jqs1/group/221108rbsdeglibrary_1_table_reformatted2.pickle"

In [ ]:
%%time
with open(pickle_filename, "wb") as f:
    pickle.dump((all_measurements, trenches), f)

In [ ]:
%%time
with open(pickle_filename, "rb") as f:
    all_measurements, trenches = pickle.load(f)

# Growth rates

In [ ]:
%%time
one_trench = all_measurements.xs(IDX[:, :, 1000, :], drop_level=False)

In [ ]:
%%time
mothers = one_trench.xs(IDX[:, :, :, 1], drop_level=False)

In [ ]:
def track_mother_cell(df):
    # groupby rois
    mothers = df.xs(IDX[:, :, :, 1], drop_level=False)
    lengths = mothers["axis_major_length"].values
    # first cell ID is 1, (0 is used as marker of non-tracked cell segment)
    mother_cell_ids = np.concatenate(([1], 1 + np.cumsum(lengths[1:] < lengths[:-1])))
    cell_ids = np.zeros(len(df), dtype=np.uint64)
    cell_ids[df.index.get_locs(IDX[:, :, :, 1])] = mother_cell_ids
    return df.assign(cell_id=cell_ids)

In [ ]:
one_trench_tracked = track_mother_cell(one_trench)
one_trench_tracked

In [ ]:
one_trench_tracked[one_trench_tracked["cell_id"] != 0].hvplot.scatter(
    "t", "axis_major_length", by="cell_id", cmap="Category20", legend=False
)

In [ ]:
%%time
all_tracked = all_measurements.groupby(["fov", "roi"], group_keys=False).progress_apply(
    track_mother_cell
)

In [ ]:
all_tracked

In [ ]:
all_tracked.xs(IDX[:, :, 3001, 1]).hvplot.scatter(
    "t", "axis_major_length", by="cell_id", cmap="Category20", legend=False
)

In [ ]:
all_growth_rates.loc[IDX[:,:,3001,:]]

In [ ]:
one_trench_onlytracked = one_trench_tracked[one_trench_tracked["cell_id"] != 0]

In [ ]:
one_cellid = one_trench_onlytracked[one_trench_onlytracked["cell_id"] == 4]
one_cellid

In [ ]:
lineage_growth_rate(one_cellid)

In [ ]:
def ols(x, y):
    num_obs = len(x)
    x_bar = x.sum() / num_obs
    y_bar = y.sum() / num_obs
    beta = (num_obs * (x * y).sum() - x.sum() * y.sum()) / (
        num_obs * (x**2).sum() - x.sum()**2
    )
    alpha = y_bar - beta * x_bar
    y_hat = alpha + beta * x
    r2 = np.sum((y_hat - y_bar) ** 2) / np.sum((y - y_bar) ** 2)
    return alpha, beta, r2

In [ ]:
def lineage_growth_rate_interval(df, min_obs=3):
    if len(df) < min_obs:
        return
    ts = df.index.get_level_values("t").values
    log_length = np.log(df["axis_major_length"].values)
    # return pd.Series(ols(ts, log_length), index=["alpha", "beta", "r2"])
    interval = pd.Interval(ts[0], ts[-1], closed="both")
    k = df.index[0]
    index = pd.MultiIndex.from_tuples(
        [(k[0], interval, k[2], df["cell_id"].iloc[0])],
        names=["fov", "t", "roi", "cell_id"],
    )
    return pd.DataFrame(
        [ols(ts, log_length)], columns=["alpha", "beta", "r2"], index=index
    )


def lineage_growth_rate(df, min_obs=3):
    if len(df) < min_obs:
        return
    ts = df.index.get_level_values("t").values
    # TODO: not necessary, but makes alpha (y-intercept) comparable between lineages
    ts -= ts.min()
    log_length = np.log(df["axis_major_length"].values)
    new_df = pd.DataFrame(
        np.repeat([ols(ts, log_length)], len(df), axis=0),
        columns=["alpha", "beta", "r2"],
        index=df.index,
    )
    return new_df.assign(cell_id=df["cell_id"]).set_index("cell_id", append=True)

In [ ]:
%%time
one_trench_onlytracked.groupby(
    ["fov", "roi", "cell_id"], group_keys=False
).progress_apply(lineage_growth_rate)

In [ ]:
%%time
all_growth_rates = all_tracked[all_tracked["cell_id"] != 0].groupby(
    ["fov", "roi", "cell_id"], group_keys=False
).progress_apply(lineage_growth_rate)

In [ ]:
all_growth_rates["r2"].hvplot.hist()

In [ ]:
# NOW: low-time-resolution growth rate for each trench (so can make 2d heatmaps)
# - dumb linear regression, batch per FOV (?) so only one LAPACK call?
# LATER: high-time-resolution growth rates grouped by genetic identity (e.g., barcode)
# - slightly (?) slower but better if we do this as an average of per-cellid growth rates
#   (that way we can filter out junk on per-cellid characteristics, r^2, etc.)

# 2D Heatmap

In [ ]:
import hvplot.xarray
import xarray as xr

In [ ]:
%%time
observable = "beta"
num_bins = 100
#measurements_subset = all_growth_rates[all_growth_rates["r2"] > 0.9]
measurements_subset = measurements_subset.reset_index()[["t", "alpha", "beta", "r2"]]
bins = np.linspace(
    measurements_subset[observable].min(),
    measurements_subset[observable].max(),
    num_bins,
)
heatmap = measurements_subset.groupby(["t"]).apply(
    lambda x: pd.Series(np.histogram(x[observable], bins=bins)[0], index=bins[:-1])
)
heatmap.columns.name = observable
heatmap = xr.DataArray(heatmap.T)

In [ ]:
heatmap.hvplot.quadmesh(
    cmap="blues",
    # logy=True,
    logz=True,
    # clim=(1, 1e4),
)